<a href="https://colab.research.google.com/github/aatika-hakim/AI/blob/main/AI_Powered_Attendance_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install langchain langgraph matplotlib streamlit pyngrok

In [18]:
# In-memory data
attendance_data = {
    "Alice": [{"date": "2024-12-01", "status": "Present"}, {"date": "2024-12-02", "status": "Absent"}],
    "Bob": [{"date": "2024-12-01", "status": "Present"}, {"date": "2024-12-02", "status": "Present"}],
}

performance_data = {
    "Alice": {"math": 85, "science": 90, "english": 88},
    "Bob": {"math": 78, "science": 85, "english": 80},
}


In [19]:
def fetch_attendance_tool(student_name):
    records = attendance_data.get(student_name, [])
    if not records:
        return f"No attendance data found for {student_name}."
    return records


In [20]:
def fetch_performance_tool(student_name):
    records = performance_data.get(student_name, {})
    if not records:
        return f"No performance data found for {student_name}."
    return records


In [21]:
!pip install --upgrade langchain langgraph # Ensure both langchain and langgraph are updated
!pip show langgraph # Verify the installation and version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 29.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7
Name: langgraph
Version: 0.2.54
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-sdk
Required-by: 


In [23]:
!pip install --upgrade langgraph # Ensures you have the latest version of langgraph
from langgraph import Agent, Tool
from langchain import OpenAI
from langchain.memory import ConversationBufferMemory

# Initialize ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")

# Attendance Tool
fetch_attendance = Tool(name="FetchAttendance", function=fetch_attendance_tool)

# Attendance Agent
attendance_agent = Agent(
    name="AttendanceAgent",
    tools=[fetch_attendance],
    llm=OpenAI(model="text-davinci-003"),
    memory=memory
)



ImportError: cannot import name 'Agent' from 'langgraph' (unknown location)

In [ ]:
# Performance Tool
fetch_performance = Tool(name="FetchPerformance", function=fetch_performance_tool)

# Performance Agent
performance_agent = Agent(
    name="PerformanceAgent",
    tools=[fetch_performance],
    llm=OpenAI(model="text-davinci-003"),
    memory=memory
)


In [ ]:
from langgraph.graph import StateGraph, State

# Define States
attendance_state = State(name="AttendanceQuery", function=attendance_agent.run)
performance_state = State(name="PerformanceQuery", function=performance_agent.run)

# Create StateGraph
student_assistance_graph = StateGraph(
    states=[attendance_state, performance_state],
    initial_state="AttendanceQuery"
)

# Add transitions
student_assistance_graph.add_transition("AttendanceQuery", "PerformanceQuery")


In [ ]:
from langgraph import Agent, Tool # Now try importing again

In [ ]:
# Example Query
query = "What is Alice's performance in math and science?"

# Choose the initial state based on the query
if "attendance" in query.lower():
    student_assistance_graph.set_initial_state("AttendanceQuery")
elif "performance" in query.lower():
    student_assistance_graph.set_initial_state("PerformanceQuery")

# Run the workflow
response = student_assistance_graph.run({"input": query})
print(response)
